## Unity to Mujoco Creature Converter

A script to convert JSON from the UTMIST's Virtual Creatures Unity project into Mujoco, which uses a nested XML file format.

In [2]:
from dataclasses import dataclass
from __future__ import annotations
import json
from pprint import pprint
from typing import List

In [3]:
# Import creature json
body_parts = {}

filename = 'creature_configs/blueprint2.json'
with open(filename) as f:
    data = json.load(f)
    for entry in data:
        id = entry["UniqueId"]
        body_parts[id] = entry    

### Breaking Down the Creature

We see that each body part is composed of the following fields:

|Field|Type|Notes|
|--|--|--|
| UniqueId | Int | A unique ID for the body part |
| TypeId | Int | ??? |
| ParentUniqueId | Int \| None | |
| Position | Vector3 | |
| LocalPosition | Vector3 | Position relative to parent |
| Rotation | Vector3 | |
| LocalRotation | Vector3 | Rotation relative to parent |
| Size | Vector3 | Size of part |
| JointType | 'hinge' \| 'fixed' \| None |  |
| JointAnchorPos | Vector3 \| None | Position of the joint (relative to child, presumeably) |
| JointAxis | Vector3 \| None | 0 or 1 for whether joint is free for each axis |
| Color | Vector3 | Hex values |

**Notes**:
- All vectors in the JSON file store their magnitudes and sometimes normalization, this seems cached and not actually useful (sometimes the normalizations have normalizations?)
- Rotation is presumeably Euler rotations

**Questions**:
- Check what `TypeID` does
- For whatever reason, `LocalPosition` is the same as `Position` even when parent position is not (0,0,0)

In [4]:
# Define an interface to interact with
@dataclass
class Vector3():
    x: float
    y: float
    z: float
    
    def as_string(self):
        return f"{self.x} {self.y} {self.z}"
    
    @staticmethod
    def from_json(json_dict):
        return Vector3(json_dict["x"], json_dict["y"], json_dict["z"])

@dataclass
class Joint():
    joint_type: str
    anchor_position: Vector3
    axis: Vector3

### Coordinate Conversions

#### Position

For most axes, we use

`2 * (unity_pos_child - unity_pos_immediate_parent)`

However, for the vertical axis, we have:

```
2 * (unity_pos_child - unity_pos_immediate_parent) + size_child - size_parent
```

We are doing this because in Unity, gameobjects are centered on the center of the bottom face, but in Mujoco, the bodies are centered in the very center of the object

#### 

In [5]:
@dataclass
class BodyPart():
    id: int
    parent: BodyPart | None
    children: List[BodyPart]
    position: Vector3
    rotation: Vector3
    size: Vector3
    color: Vector3
    joint: Joint | None
    
    ################## Coordinate Conversions ##################
    
    def mujoco_position(self):
        parent_position = self.parent.position if self.parent else Vector3(0, 0, 0)
        parent_size = self.parent.size if self.parent else Vector3(0, 0, 0)
        
        mujoco_x = 2 * (self.position.x - parent_position.x)
        mujoco_y = 2 * (self.position.z - parent_position.z)
        mujoco_z = 2 * (self.position.y - parent_position.y) + self.size.y - parent_size.y
        
        return Vector3(mujoco_x, mujoco_y, mujoco_z)
    
    def mujoco_size(self):
        return Vector3(self.size.x, self.size.z, self.size.y)
    
    def mujoco_joint_position(self):
        if (self.joint):
            return Vector3(-self.joint.anchor_position.x, -self.joint.anchor_position.z, -self.joint.anchor_position.y)
    
    ############################################################
    
    def print(self, layer=0):
        def layered_print(str):
            prefix = '    ' * layer
            print(f"{prefix}{str}")
        
        layered_print(f"\033[1m\033[34mBody Part {self.id}\033[0m")
        layered_print(f"ID:         {self.id}")
        layered_print(f"Position:   {self.mujoco_position().as_string()}")
        layered_print(f"Rotation:   {self.rotation.as_string()}")
        layered_print(f"Size:       {self.mujoco_size().as_string()}")
        
        if self.joint:
            layered_print(f"Joint:      Joint Type: {self.joint.joint_type}")
            layered_print(f"            Joint Anchor: {self.mujoco_joint_position().as_string()}")
            layered_print(f"            Joint Axis: {self.joint.axis.as_string()}")
        else:
            layered_print(f"Joint:      None")
        
        for child in self.children:
            child.print(layer + 1)
    
    @staticmethod 
    def from_json(json_dict, parent=None):
        joint = Joint(
            joint_type=json_dict["JointType"],
            anchor_position=Vector3.from_json(json_dict["JointAnchorPos"]),
            axis=Vector3.from_json(json_dict["JointAxis"])
        ) if json_dict["JointType"] else None
        
        return BodyPart(
            id=json_dict["UniqueId"],
            parent=parent,
            children=[],
            position=Vector3.from_json(json_dict["Position"]),
            rotation=Vector3.from_json(json_dict["Rotation"]),
            size=Vector3.from_json(json_dict["Size"]),
            color=Vector3.from_json(json_dict["Color"]),
            joint=joint
        )


In [6]:
root = BodyPart.from_json(body_parts[0])

def recursively_assemble_creature(root: BodyPart, body_parts_dict):
    for body_part in body_parts_dict.values():
        if body_part["ParentUniqueId"] == root.id:
            root.children.append(BodyPart.from_json(body_part, root))
            recursively_assemble_creature(root.children[-1], body_parts_dict)
            
recursively_assemble_creature(root, body_parts)

In [7]:
root.print()

Body Part 0
ID:         0
Position:   0.0 0.0 1.3171674
Rotation:   0.0 0.0 0.0
Size:       0.7817855 0.436341643 1.3171674
Joint:      None
    Body Part 1
    ID:         1
    Position:   -0.746078492 0.13879776 1.736679731
    Rotation:   0.0008985509 0.00109280727 0.004466458
    Size:       0.595344 0.388579845 0.419512331
    Joint:      Joint Type: hinge
                Joint Anchor: 0.477163136 -0.159047112 -1.0
                Joint Axis: 0.0 1.0 0.0
    Body Part 2
    ID:         2
    Position:   0.781785488 -0.12621116 -0.4193217760000001
    Rotation:   0.000849346863 0.00307855778 0.0009179028
    Size:       0.165260255 0.482132822 0.5905807
    Joint:      Joint Type: hinge
                Joint Anchor: -0.49999997 0.144624248 -0.116638534
                Joint Axis: 0.0 0.0 1.0
        Body Part 3
        ID:         3
        Position:   -0.08037185600000007 0.482152932 1.013563076
        Rotation:   0.000849346863 0.00307855778 0.0009179028
        Size:       0.1